In [4]:
bronze_output = ""
silver_data = ""

StatementMeta(synonly, 16, 5, Finished, Available, Finished)

In [27]:
'''
from datetime import date, timedelta

#Remove this before running datafactory
start_date = date.today() - timedelta(1)
gold_adls = "abfss://gold@storage999.dfs.core.windows.net/"
silver_adls = "abfss://silver@storage999.dfs.core.windows.net/"
silver_data = f"{silver_adls}earthquake_events_silver/"
'''

StatementMeta(hourpool, 13, 31, Finished, Available, Finished)

In [28]:
silver_data

StatementMeta(hourpool, 13, 32, Finished, Available, Finished)

'abfss://silver@storage999.dfs.core.windows.net/earthquake_events_silver/'

In [29]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType
# Ensure the below library is installed on your cluster
import reverse_geocoder as rg
from datetime import date, timedelta

StatementMeta(hourpool, 13, 33, Finished, Available, Finished)

In [30]:
df = spark.read.parquet(silver_data).filter(col('time') > start_date)

StatementMeta(hourpool, 13, 34, Finished, Available, Finished)

In [31]:
df = df.limit(10) # added to speed up processings as during testing it was proving a bottleneck
# The problem is caused by the Python UDF (reverse_geocoder) being a bottleneck due to its non-parallel nature and high computational cost per task

StatementMeta(hourpool, 13, 35, Finished, Available, Finished)

In [32]:
def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_country_details(48.8588443, 2.2943506)
    'FR'
    """
    try:
        coordinates = (float(lat), float(lon))
        result = rg.search(coordinates)[0].get('cc')
        print(f"Processed coordinates: {coordinates} -> {result}")
        return result
    except Exception as e:
        print(f"Error processing coordinates: {lat}, {lon} -> {str(e)}")
        return None

StatementMeta(hourpool, 13, 36, Finished, Available, Finished)

In [33]:
# registering the udfs so they can be used on spark dataframes
get_country_code_udf = udf(get_country_code, StringType())

StatementMeta(hourpool, 13, 37, Finished, Available, Finished)

In [34]:
get_country_code(48.8588443, 2.2943506)

StatementMeta(hourpool, 13, 38, Finished, Available, Finished)

Processed coordinates: (48.8588443, 2.2943506) -> FR


'FR'

In [35]:
# adding country_code and city attributes
df_with_location = \
                df.\
                    withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))

StatementMeta(hourpool, 13, 39, Finished, Available, Finished)

In [36]:
df.printSchema()

StatementMeta(hourpool, 13, 40, Finished, Available, Finished)

root
 |-- id: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- elevation: double (nullable = true)
 |-- title: string (nullable = true)
 |-- place_description: string (nullable = true)
 |-- sig: long (nullable = true)
 |-- mag: double (nullable = true)
 |-- magType: string (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- updated: timestamp (nullable = true)



In [37]:
df_with_location.printSchema()

StatementMeta(hourpool, 13, 41, Finished, Available, Finished)

root
 |-- id: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- elevation: double (nullable = true)
 |-- title: string (nullable = true)
 |-- place_description: string (nullable = true)
 |-- sig: long (nullable = true)
 |-- mag: double (nullable = true)
 |-- magType: string (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- updated: timestamp (nullable = true)
 |-- country_code: string (nullable = true)



In [38]:
# adding significance classification
df_with_location_sig_class = \
                            df_with_location.\
                                withColumn('sig_class', 
                                            when(col("sig") < 100, "Low").\
                                            when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                            otherwise("High")
                                            )

StatementMeta(hourpool, 13, 42, Finished, Available, Finished)

In [39]:
df_with_location_sig_class.printSchema()

StatementMeta(hourpool, 13, 43, Finished, Available, Finished)

root
 |-- id: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- elevation: double (nullable = true)
 |-- title: string (nullable = true)
 |-- place_description: string (nullable = true)
 |-- sig: long (nullable = true)
 |-- mag: double (nullable = true)
 |-- magType: string (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- updated: timestamp (nullable = true)
 |-- country_code: string (nullable = true)
 |-- sig_class: string (nullable = false)



In [40]:
# Save the transformed DataFrame to the Silver container
gold_output_path = f"{gold_adls}earthquake_events_gold/"

StatementMeta(hourpool, 13, 44, Finished, Available, Finished)

In [41]:
# Append DataFrame to Silver container in Parquet format
df_with_location_sig_class.write.mode('append').parquet(gold_output_path)

StatementMeta(hourpool, 13, 45, Finished, Available, Finished)